In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast

In [3]:
def shorten(input_string):
    """Return the first 20 characters of the title."""
    return input_string[:20]

### How much Printify costs have we been spending each month?

In [4]:
printify_df = pd.read_csv("../data/all_shops_orders.csv")
printify_df['Metadata'] = printify_df['Metadata'].apply(ast.literal_eval)
printify_df['Shop Order ID'] = printify_df['Metadata'].apply(lambda x: x.get('shop_order_id',0) if isinstance(x,dict) else 0)
printify_df['Total Price'] = pd.to_numeric(printify_df['Total Price']/100, errors='coerce')
printify_df['Total Shipping'] = pd.to_numeric(printify_df['Total Shipping']/100, errors='coerce')
printify_df['Shop Order ID'] = printify_df['Shop Order ID'].apply(str)

print(f"there are {printify_df.shape[0]} printify transactions")

there are 6724 printify transactions


In [5]:
printify_df2 = printify_df[printify_df['Status']!='canceled']
printify_df2 = printify_df2[printify_df2['Metadata'].apply(lambda x: x['order_type']!='sample')]
printify_df2['Final Cost'] = printify_df2['Total Price'] + printify_df2['Total Shipping']
printify_df2['Created At'] = pd.to_datetime(printify_df2['Created At'])
printify_df2['Month/Year'] = printify_df2['Created At'].dt.to_period('M')


C:\Users\arsen\AppData\Local\Temp\ipykernel_1820\1581532336.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  printify_df2['Month/Year'] = printify_df2['Created At'].dt.to_period('M')


In [6]:
printify_df2.head()

,Shop Title,Order ID,App Order ID,Shop ID,First Name,Last Name,Email,Address,City,Region,...,Total Price,Total Shipping,Status,Created At,Line Items,SKU,Metadata,Shop Order ID,Final Cost,Month/Year
1,DTC Etsy,67747e30eed3d425220a8d02,1.329733e+07,13297332,john,becker,danjillbecker@aol.com,1502 E Quail Ridge Ct,Andover,KS,...,7.56,5.15,on-hold,2024-12-31 23:28:48+00:00,"Fried Chicken 420 Shirt (Qty: 1, Variant: Blac...",13618552896815833100,"{'order_type': 'api', 'shop_order_id': '355858...",3558589469,12.71,2024-12
2,DTC Etsy,6774458c90d0140003053f1c,1.329733e+07,13297332,Hannah,Marie Martin,slappingwithsirens@gmail.com,154 Pearces Rd,HATTIESBURG,MS,...,7.56,5.15,on-hold,2024-12-31 19:27:08+00:00,"rippit bong frog (Qty: 1, Variant: Black / L)",56339870238510617188,"{'order_type': 'api', 'shop_order_id': '355845...",3558459729,12.71,2024-12
3,DTC Etsy,67740d4352a2e3d452040987,1.329733e+07,13297332,Kira,Just De La Paisieres,kjjust@t-online.de,Am Weißen Kreuz 5,Königswinter,KÖNIGSWINTER,...,6.43,3.79,in-production,2024-12-31 15:26:59+00:00,"EUROPE Minye | Cursed Shirt, Weirdcore, Shirt ...",35821319389606437256,"{'order_type': 'api', 'shop_order_id': '355832...",3558321433,10.22,2024-12
4,DTC Etsy,67739cad40916a588904f445,1.329733e+07,13297332,Dena,Valesky,denavalesky@gmail.com,106 Muirfield Drive,Pittsburgh,PA,...,15.02,7.55,fulfilled,2024-12-31 07:26:37+00:00,"rippit bong frog (Qty: 2, Variant: Black / L)",26971594069434489886,"{'order_type': 'api', 'shop_order_id': '355803...",3558035383,22.57,2024-12
5,DTC Etsy,67736462f508356b1d045902,1.329733e+07,13297332,Alex,Graham,alexgraham574@icloud.com,38 Castleton,"Templepatrick, Antrim and Newtownabbey",NIR,...,7.52,3.49,in-production,2024-12-31 03:26:26+00:00,"UK 3 - Minye TShirt | Cursed Shirt, Weirdcore,...",24676907806028478631,"{'order_type': 'api', 'shop_order_id': '355237...",3552376954,11.01,2024-12


## Create Cost Table

In [10]:
# Extract 'month/year' from the 'Created At' column
printify_df2['Month/Year'] = printify_df2['Created At'].dt.to_period('M')

# Pivot the table
pivot_table = printify_df2.pivot_table(
    index='Month/Year',        # Rows
    columns='Shop Title',      # Columns
    values='Final Cost',      # Values
    aggfunc='sum',             # Aggregation function
    fill_value=0               # Fill missing values with 0
)



# Add a 'Total' column for each month
pivot_table['Total'] = pivot_table.sum(axis=1)

# Calculate the grand total of the 'Total' column
grand_total = pivot_table['Total'].sum()

# Round all values to 2 decimal places
pivot_table = pivot_table.round(2)

# Reset index for better readability
pivot_table = pivot_table.reset_index()

print(pivot_table)
print(f"Grand total for the year is {grand_total}")

Shop Title Month/Year  DTC Amazon  DTC Etsy  DTC TikTok     Total
0             2023-12        0.00     57.18        0.00     57.18
1             2024-01        0.00    780.88        0.00    780.88
2             2024-02        0.00    831.63        0.00    831.63
3             2024-03        0.00    255.25        0.00    255.25
4             2024-04        0.00    280.43        0.00    280.43
5             2024-05        0.00    292.27        0.00    292.27
6             2024-06        0.00    386.78        0.00    386.78
7             2024-07        0.00    978.99       61.05   1040.04
8             2024-08        0.00   1464.85     8508.86   9973.71
9             2024-09        0.00   2883.88    13501.85  16385.73
10            2024-10       10.92   2692.84     3887.24   6591.00
11            2024-11      271.91   7570.12    14944.53  22786.56
12            2024-12      819.07  15054.59    18299.40  34173.06
Grand total for the year is 93834.51999999999


C:\Users\arsen\AppData\Local\Temp\ipykernel_1820\4089750641.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  printify_df2['Month/Year'] = printify_df2['Created At'].dt.to_period('M')


In [11]:
pivot_table.to_csv('../app/static/cost_table.csv', index=False)